# Data Scraper

In [664]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
from urllib.request import urlopen
import matplotlib.pyplot as plt

In [2]:
requestHead = {"User-Agent": "Chrome/47.0.2526.111"}

In [11]:
def percentDist(row):
    makeMiss = row[dist].split('/')
    makeMiss = [int(m) for m in makeMiss]
    if makeMiss[1] != 0:
        pct = makeMiss[0]/makeMiss[1]
    else:
        pct = 0.
    return pct*100

In [8]:
def teamID(row,S):
    if S < 5:
        teamIDs = {'1':'BAL','2':'YKW','3':'COL','4':'ARI','5':'OCO','6':'SJS','7':'PHI','8':'LV','9':'CHI','10':'AUS'}
    elif S == 5:
        teamIDs = {'1':'BAL','2':'YKW','3':'COL','4':'AZ','5':'OCO','6':'SJS','7':'PHI','8':'LV','9':'CHI','10':'AUS'}
    else:
        teamIDs = {'1':'BAL','2':'YKW','3':'COL','4':'AZ','5':'OCO','6':'SJS','7':'PHI','8':'NO','9':'CHI','10':'AUS'}
    
    for ID,team in teamIDs.items():
        if row.Team == team:
            return int(ID)

In [23]:
def standID(S,row):
    if S <= 5:
        teamIDs = {'1':'Baltimore Hawks','2':'Yellowknife Wraiths','3':'Colorado Yeti','4':'Arizona Outlaws','5':'Orange County Otters',
                   '6':'San Jose SaberCats','7':'Philadelphia Liberty','8':'Las Vegas Legion','9':'Chicago Butchers','10':'Austin Copperheads'}
    else:
        teamIDs = {'1':'Baltimore Hawks','2':'Yellowknife Wraiths','3':'Colorado Yeti','4':'Arizona Outlaws','5':'Orange County Otters',
                   '6':'San Jose SaberCats','7':'Philadelphia Liberty','8':'New Orleans Second Line','9':'Chicago Butchers','10':'Austin Copperheads'}
        
    for ID,team in teamIDs.items():
        if row.Team == team:
            return int(ID)

In [639]:
def getSeason(num,stat='Kicking'):
    if num < 10:
        strnum = '0%s'%num
    else:
        strnum = str(num)
    
    url = "http://sim-football.com/indexes/NSFLS"+strnum+"/League%sStats.html"%stat
    page = requests.get(url, headers = requestHead)
    soup = BeautifulSoup(page.text,'html.parser') 
    
    kickTable = soup.find_all('table', class_ = 'tablesorter')[0]
    kickCols=['Name','Pos','Team','G','XPM','XPA','XPpct','FGM','FGA','FGpct','<20','20-29','30-39','40-49','50+','Lng']
    testList = []
    for row in kickTable.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) > 0:
            testList.append([t.text for t in cols])
    kickDF = pd.DataFrame(testList,columns=[t.text for t in kickTable.find_all('tr')[0].find_all('th')])
    for i in kickDF.columns:
        try:
            kickDF[i] = pd.to_numeric(kickDF[i])
        except ValueError:
            continue
    kickDF['teamID'] = kickDF.apply(lambda row: teamID(row,int(strnum)),axis=1)
#     for dist in ['<20','20-29','30-39','40-49','50+']:
#         kickDF[dist] = kickDF.apply(lambda row: percentDist(row),axis=1)
    kickerDF = pd.DataFrame(columns = ['Player','Ovr', 'Str', 'Agi', 'Arm', 'Int', 'Acc', 'Tck', 'Spe','Hnd', 'PBl', 'RBl', 'KDi', 'KAc', 'End'])
    for p in range(len(kickDF)):
    #     print(p)
        playerRow = kickDF.loc[p]
        playerName = playerRow['Name']
        playerTeam = playerRow['teamID']

        urlTeam = 'http://sim-football.com/indexes/NSFLS%s/Teams/%s.html'%(strnum,str(playerTeam))
        pageTeam = requests.get(urlTeam, headers = requestHead)
        soupTeam = BeautifulSoup(pageTeam.text,'html.parser')
        rosterSoup = soupTeam.find_all(id = 'tabs-2')[0]
        rosterTable = rosterSoup.find_all('table',class_='Grid')[0]

        playerList = []
        for row in rosterTable.find_all('tr'):
            cols = row.find_all('td')
            if len(cols) > 0:
                playerList.append([t.text for t in cols])
        teamDF = pd.DataFrame(playerList)
        teamDF.columns = teamDF.iloc[0]
        teamDF = teamDF[1:]
        try:
            playerDF = teamDF[teamDF['Player'] == playerName][['Player','Ovr', 'Str', 'Agi', 'Arm', 'Int', 'Acc', 'Tck', 'Spe','Hnd', 'PBl', 'RBl', 'KDi', 'KAc', 'End']]
            playerDF.index = [p]
            kickerDF = pd.concat([kickerDF,playerDF])
        except ValueError:
            kickDF = kickDF.drop([p],axis=0)
    for i in ['Ovr', 'Str', 'Agi', 'Arm', 'Int', 'Acc', 'Tck', 'Spe','Hnd', 'PBl', 'RBl', 'KDi', 'KAc', 'End']:
        kickerDF[i] = pd.to_numeric(kickerDF[i])
    kickerDF.columns = ['Player','OVR','STR','AGI','ARM','INT','ACC','TCK','SPE','HND','PBl','RBl','KDi','KAc','END']
    kickDF = pd.concat([kickDF,kickerDF],axis=1)
    kickDF = kickDF.sort_values('teamID').set_index('teamID')
    urlStand = 'http://sim-football.com/indexes/NSFLS%s/Index.html'%strnum
    pageStand = requests.get(urlStand,headers=requestHead)
    soupStand = BeautifulSoup(pageStand.text,'html.parser')
    standList = soupStand.find_all('table', class_='Grid')
    NSFCstand = standList[0]
    ASFCstand = standList[1]
    teamList = []
    for stand in [NSFCstand,ASFCstand]:
        for row in stand.find_all('tr'):
            cols = row.find_all('td')
            links = row.find_all('a')
            if len(links) > 0:
    #             print(links)
    #             print(cols)
                teamList.append([t.text for t in cols])
    standDF = pd.DataFrame(teamList,columns=['Team','W','L','T','Pct','PF','PA','Home','Road','Conf','Div','Str'])
    for i in ['W','L','T','Pct','PF','PA']:
        standDF[i] = pd.to_numeric(standDF[i])
    standDF['teamID'] = standDF.apply(lambda row: standID(int(strnum),row),axis=1)
    standDF = standDF.sort_values('teamID').set_index('teamID')
    standDF = standDF[['W','L','T','Pct','PF','PA']]
    standDF.columns = ['W','L','T','WPct','PF','PA']
    kickDF = pd.concat([kickDF,standDF],axis=1)
    kickDF['S'] = int(strnum)
    
    return kickDF

In [661]:
def getAllStats(statistic):
    statDF = getSeason(1,stat=statistic)
    progBar = tqdm(total=19,desc='Season')
    progBar.update(1)
    for s in range(2,20):
        statDF = pd.concat([statDF,getSeason(s,stat=statistic)],sort=True)
        progBar.update(1)
    
    statDF2 = statDF.reset_index().dropna()
    statDF2.to_csv('%sDF.csv'%(statistic))
    return statDF

In [665]:
kickDF = getAllStats("Kicking")

In [669]:
statList = ['Passing','Rushing','Receiving','Defensive','Kicking','Punting','OffensiveLine','SpecialTeams']

In [670]:
for stat in statList:
    print(stat)
    getAllStats(stat)

Passing


Rushing


Receiving


Defensive


Kicking


Punting


OffensiveLine


SpecialTeams
